In [2]:
# useing this variable for saving models and summaries
exp_name = "CNN1014"

## Importing Libraries

In [3]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import pickle 

import sys
sys.path.append('../../../src')

In [4]:
import models.CNN_model as CNN_model
import training
import evaluation.nn_eval as nn_eval

## Loading synthetic data

In [5]:
#loading data
data_path= '/data2/data/zebrahim/synthetic_dataset/data_8192_1000_3_4_4_.03_.03_.2' + 'processed_data.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']
 

In [6]:
#index of changes for test dataset
gt_test_idx_changes = index_of_changes[900:1000]

## Model

In [7]:
# Model parameters
n_variables = 12
learning_rate = 0.001
batch_size = 10

convolution_levels = 6    #number of convolution to have in the network
pooling_stride = [16, 2, 1, 1, 1, 1]
weights_shape = [[7, n_variables, 32],
                 [7, 32, 32],
                 [7, 32, 32],
                 [7, 32, 32],
                 [7, 32, 32],
                 [7, 32, 32]]
bias_shape = [32, 32, 32, 32, 32, 32]
activation = tf.nn.relu
n_classes = 1

##positive weight for weighted cross entropy
s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()

In [8]:
print pos_weight

3.18601822124


In [9]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [10]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 

In [11]:
model = CNN_model.Model(n_variables = n_variables,
                        learning_rate = learning_rate,
                        n_classes = n_classes)

In [12]:
model.build_conv_net(pos_weight = pos_weight,
                     learning_rate = learning_rate,
                     convolution_levels = convolution_levels, 
                     pooling_stride = pooling_stride,
                     weights_shape = weights_shape,
                     bias_shape = bias_shape,
                     n_classes = n_classes,
                     activation = activation)

In [13]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = validation_data,
                    ground_truth_validation = validation_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0)

In [14]:
trainer.train(max_iter = 10000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Tarin Iter 0, auc= 0.224924, fmeasure_max= 0.369001, precision= 0.230360, recall= 0.926786
Train Random guess:  , auc= 0.208739fmeasure_max= 0.359320, precision= 0.219007, recall= 1.000000
Validation Iter 0, auc= 0.304050, fmeasure_max= 0.378865, precision= 0.245436, recall= 0.830189
Validation Random guess:  auc= 0.239058, fmeasure_max= 0.371338, precision= 0.228002, recall= 1.000000
Minibatch Loss= 1.039126
Tarin Iter 100, auc= 0.689551, fmeasure_max= 0.679558, precision= 0.760825, recall= 0.613977
Train Random guess:  , auc= 0.241214fmeasure_max= 0.380500, precision= 0.234949, recall= 1.000000
Validation Iter 100, auc= 0.737663, fmeasure_max= 0.770903, precision= 0.752039, recall= 0.790738
Validation Random guess:  auc= 0.222662, fmeasure_max= 0.371410, precision= 0.228146, recall= 0.998285
Minibatch Loss= 0.847686
Tarin Iter 200, auc= 0.794610, fmeasure_max= 0.735088, precision= 0.748214, recall= 0.722414
Train Random guess:  , auc= 0.242387fmeasure_max= 0.369496, precision= 0.2267

In [ ]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

In [ ]:
#load the model
saver = tf.train.Saver()
saver.restore(sess, "../models/model" + exp_name + "/model")

In [16]:
#This is the model evaluation on test dataset
probability_of_pos= nn_eval.evaluate_model (model, sess, test_data, test_gt)

Testing auc = 0.916741 F1_max = 0.845877 prec = 0.858684 rec = 0.833446
Test Random guess: auc= 0.228111 fmeasure_max= 0.375635 , precision= 0.231250 , recall= 1.000000


In [ ]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [ ]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})